In [1]:
# Initial imports
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score


In [2]:
Combined_Data = pd.read_csv("Resources/Cleaned CSVs/Combined_Data.csv")

Combined_Data.drop('Tm', axis=1, inplace=True)

# Combined_Data.set_index('Tm', inplace=True)

In [3]:
corr_data = Combined_Data.corr()['playoffs']

low_corr_df = pd.DataFrame(corr_data.loc[(corr_data <= 0.11) & (corr_data >= -0.11)])
low_corr_df.rename(columns={'playoffs': 'correlation'}, inplace=True)

index_values = low_corr_df.index.tolist()

index_values

['3B', 'BK', 'CG_y', 'Ch', 'GF', 'HBP_x', 'SB', 'cSho']

In [4]:
columns_to_keep = ['3B', 'BK', 'CG_y', 'Ch', 'GF', 'HBP_x', 'SB', 'cSho', 'playoffs']

focused_df = Combined_Data[columns_to_keep]

focused_df

,3B,BK,CG_y,Ch,GF,HBP_x,SB,cSho,playoffs
0,50.0,5.0,2.0,6150.0,160.0,52.0,79.0,1.0,0
1,29.0,8.0,2.0,5889.0,160.0,66.0,90.0,1.0,1
2,15.0,8.0,2.0,5934.0,160.0,57.0,81.0,0.0,0
3,31.0,3.0,2.0,5819.0,160.0,55.0,125.0,0.0,1
4,34.0,3.0,1.0,6208.0,162.0,78.0,66.0,0.0,1
...,...,...,...,...,...,...,...,...,...
145,12.0,7.0,0.0,5944.0,162.0,70.0,101.0,0.0,0
146,24.0,1.0,0.0,5802.0,162.0,95.0,160.0,0.0,1
147,18.0,4.0,3.0,5789.0,159.0,53.0,79.0,1.0,1
148,19.0,12.0,1.0,5733.0,161.0,69.0,99.0,1.0,1


In [5]:
# Separate the Features (X) from the Target (y)

y = focused_df["playoffs"]
X = focused_df.drop(columns="playoffs")

In [6]:
# Splitting into Train and Test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (112, 8)
X_test shape: (38, 8)


In [7]:
# Create the StandardScaler instance
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Instantiate the model with k = 6 neighbors
model = KNeighborsClassifier(n_neighbors=6)

In [9]:
# Train the model
model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=6)

In [10]:
# Create predictions
y_pred = model.predict(X_test_scaled)

# Review the predictions
y_pred

array([1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_pred, y_test)

In [12]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_pred, y_test))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,20,5
Actual 1,10,3


Accuracy Score : 0.6052631578947368
Classification Report
              precision    recall  f1-score   support

           0       0.80      0.67      0.73        30
           1       0.23      0.38      0.29         8

    accuracy                           0.61        38
   macro avg       0.52      0.52      0.51        38
weighted avg       0.68      0.61      0.63        38

